1、数据的存储
举例1: 从TXT文档中加载数据，向量化后存储到Chroma数据库

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader

import os
import dotenv

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

loader = TextLoader(
    file_path="./asset/load/09-ai1.txt",
    encoding="utf-8",
)

docs = loader.load()

# 步骤2: 创建文本拆分器, 并拆分文档
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

splitter_docs = text_splitter.split_documents(docs)

print(len(splitter_docs))

# 步骤3: 创建嵌入模型
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# 步骤4: 将文档及嵌入模型传入到Chroma相关的结构中, 进行数据的存储
db = Chroma.from_documents(
    documents=splitter_docs,
    embedding=embedding_model,
)

1、当前的数据到底存储在哪里了?
如果我们使用的 from_documents() 中没有显式的指定存储位置的话，则将当前的数据存储在内存中，并缓存起来。如果需要指明具体的存储位置，需要设置参数 persist_directory 的值。

In [ ]:
db1 = Chroma.from_documents(
    documents=splitter_docs,
    embedding=embedding_model,
    persist_directory="./asset/chroma-4",
)

2、需要明确，在向量数据库中，不仅存储了数据（或文档）的向量，而且还存储了数据（或文档）本身。

演示一下：检索的需求

In [ ]:
query = "人工智能的核心技术有哪些呢？"

docs = db.similarity_search(query)

print(docs[0].page_content)

In [ ]:
# 1.导入相关依赖
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

# 2.定义文档
raw_documents = [
    Document(
        page_content="葡萄是一种常见的水果,属于葡萄科葡萄属植物。它的果实呈圆形或椭圆形,颜色有绿色、紫色、红色等多种。葡萄富含维生素C和抗氧化物质,可以直接食用或酿造成葡萄酒。",
        metadata={"source": "水果", "type": "植物"}
    ),
    Document(
        page_content="白菜是十字花科蔬菜,原产于中国北方。它的叶片层层包裹形成紧密的球状,口感清脆微甜。白菜富含膳食纤维和维生素K,常用于制作泡菜、炒菜或煮汤。",
        metadata={"source": "蔬菜", "type": "植物"}
    ),
    Document(
        page_content="狗是人类最早驯化的动物之一,属于犬科。它们具有高度社会性,能理解人类情绪,常被用作宠物、导盲犬或警犬,不同品种的狗在体型、毛色和性格上有很多差异。",
        metadata={"source": "动物", "type": "哺乳动物"}
    ),
    # ------------------ 第二张图中的文档 ------------------
    Document(
        page_content="猫是小型肉食性哺乳动物,性格独立但也能与人类建立亲密关系。它们夜晚视力极强,擅长捕猎老鼠。家猫的品种包括波斯猫、暹罗猫、毛色和花纹多样。",
        metadata={"source": "动物", "type": "哺乳动物"}
    ),
    Document(
        page_content="人类是地球上最具智慧的生物,属于灵长目人科。现代人类(智人)拥有高度发达的大脑,创造了语言、工具和文明。人类的平均寿命约70-80年,分布在全世界各地。",
        metadata={"source": "生物", "type": "灵长类"}
    ),
    Document(
        page_content="太阳是太阳系的中心恒星,直径约139万公里,主要由氢和氦组成。它通过核聚变反应产生能量,为地球提供光和热。太阳活动周期约为11年,会影响地球气候。",
        metadata={"source": "天文", "type": "恒星"}
    ),
    Document(
        page_content="长城是中国古代的军事防御工程,总长是超过2万公里。它始建于春秋战国时期,秦朝连接各段,明朝大规模修缮。长城是世界文化遗产和人类建筑奇迹。",
        metadata={"source": "历史", "type": "建筑"}
    ),
    Document(
        page_content="量子力学是研究微观粒子运动的物理学分支。它提出了波粒二象性、测不准原理等概念,彻底改变了人类对物质世界的认知。量子计算机正基于这一理论发展而来。",
        metadata={"source": "物理", "type": "科学"}
    ),
    Document(
        page_content="《红楼梦》是中国古典文学四大名著之一,作者是曹雪芹,小说以贾、史、王、薛四大族的兴衰为背景,描绘了贾宝玉与林黛玉的爱情悲剧,反映了封建社会的种种矛盾。",
        metadata={"source": "文学", "type": "小说"}
    ),
    Document(
        page_content="新冠病毒(SARS-CoV-2)是一种可引起呼吸道疾病的冠状病毒。它通过飞沫传播,主要症状包括发热、咳嗽、乏力。疫苗和戴口罩是有效的预防措施。",
        metadata={"source": "医学", "type": "病毒"}
    )
]

# 3.创建嵌入模型
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

# 4.创建向量数据库
db = Chroma.from_documents(
    documents=raw_documents,
    embedding=embedding,
    persist_directory="./asset/chroma-3",
)

In [ ]:
# 5. 检索示例 (返回前3个最相关结果)
query = "哺乳动物"
docs = db.similarity_search(query, k=3) # k=3表示返回3个最相关文档
print(f"查询: '{query}' 的结果:")

for i, doc in enumerate(docs, 1):
    print(f"\n结果 {i}:")
    print(f"内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")

In [ ]:
# ② 支持直接对问题向量查询 (similarity_search_by_vector)
query = "哺乳动物"
embedding_vector = embedding.embed_query(query)

docs = db.similarity_search_by_vector(embedding_vector, k=3)

print(f"查询: '{query}' 的结果:")
for i, doc in enumerate(docs, 1):
    print(f"\n结果 {i}:")
    print(f"内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")

# ③ 相似性检索, 支持过滤元数据 (filter)


In [ ]:
query = "哺乳动物"

docs = db.similarity_search(
    query=query,
    k=3,
    filter={"source": "动物"}
)

for i, doc in enumerate(docs, 1):
    print(f"\n结果 {i}:")
    print(f"内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")

# ④ 通过L2距离分数进行搜索 (similarity_search_with_score)


In [ ]:
docs = db.similarity_search_with_score(
    "量子力学是什么?"
)
for doc, score in docs:
    print(f"[L2距离得分={score:.3f}] {doc.page_content} {doc.metadata}")

说明: 分数值越小，检索到的文档和问题越相似。分数值取值范围: $[0, \text{正无穷}]$

# ⑤ 通过余弦相似度分数进行搜索 (_similarity_search_with_relevance_scores)

In [ ]:
docs = db._similarity_search_with_relevance_scores(
    "量子力学是什么?"
)
for doc, score in docs:
    print(f"[余弦相似度得分={score:.3f}] {doc.page_content} {doc.metadata}")

说明：分数阈值越接近 1 (上限) , 检索到的文档越和问题相似。

MMR (最大边缘相关性, max_marginal_relevance_search)
MMR 是一种平衡相关性 和 多样性 的检索策略, 避免返回高度相似的冗余结果。


In [ ]:


docs = db.max_marginal_relevance_search(
    query="量子力学是什么",
    lambda_mult=0.8, # 侧重相似性
)

print("关于【量子力学是什么】的搜索结果:")
print("=" * 50)
for i, doc in enumerate(docs):
    print(f"\n结果 {i+1}:")
    print(f"内容: {doc.page_content}")
    print(f"标签: {', '.join(f'{k}={v}' for k, v in doc.metadata.items())}")

 lambda_mult 参数值介于 0 到 1 之间，用于确定结果之间的多样性程度，其中 0 对应最大多样性，1 对应最小多样性。默认值为 0.5。